<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [26]</a>'.</span>

In [1]:
import pandas as pd 
import numpy as np
import os
from subprocess import check_output
import shutil
import rdflib
from multiprocessing import Process, Manager, Queue
import multiprocessing

from rule import *
from amie import *
from experiment import *

In [2]:
root_source_FB = "./../../FB15k_mail/"
root_source_DB = "./../../DB15k_mail/"

root_data_FB = "./../../Data_mail/FB_ST_"
root_data_DB = "./../../Data_mail/DB_ST_"

# FB15K

In [3]:
data = open(root_source_FB+"numericals.txt", "r")

numerical_predicate = set()

for predicate in data:
    numerical_predicate.add(predicate.split("\n")[0])
    
data.close()

In [4]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [5]:
def give_group(value, groups):
    for i, value_group in enumerate(groups):
        if value < value_group:
            return i
    return i+1

def write_file(X, f, groups):
    f.write(f"<http:{X['Subject']}>\t{X['Predicate']}\t<http:/{give_group(X['Object'], groups)}>\n")

In [6]:
data = open(root_source_FB+"train.txt", "r")
f = open(root_data_FB+"train.tsv", "w")

dic_predicate = {}

for line in data: 
    line_split = line.split("\n")[0].split("\t")
    if line_split[1] in numerical_predicate:
        line_split[2] = float(line_split[2]) 
        line_split = tuple(line_split)
        if line_split[1] in dic_predicate.keys():
            dic_predicate[line_split[1]].add(line_split)
        else : 
            dic_predicate[line_split[1]] = {line_split}
    else:
        f.write('<http:'+line_split[0]+'>\t<http:'+line_split[1]+'>\t'+'<http:'+line_split[2]+'>\n')
    
data.close()
f.close()

In [7]:
f = open(root_data_FB+"train.tsv", "a")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    tp_df.apply(write_file, args=(f, tp_df_describe), axis=1)

f.close()

In [8]:
res = check_output(f'java -jar ./../amie3.jar {root_data_FB+"train.tsv"}', shell=True)

res_parsed = parse_amie(res)

Using the default schema relations


In [9]:
len(res_parsed)

46163

In [10]:
# res_parsed

### Number of numericals in the rules

In [11]:
num_f = open("ST_num_rules.txt", "w")
symb_f = open("ST_symb_rules.txt", "w")

def predicate_is_numerical(atom, numerical_predicate):
    return atom.predicate in numerical_predicate

rule_with_numerical_in_hyp = 0
rule_with_numerical_in_conc = 0
rule_with_numerical = 0

for rule in res_parsed:
    num = False
    for hyp in rule.hypotheses:
        if predicate_is_numerical(hyp, numerical_predicate):
            rule_with_numerical_in_hyp+=1
            num = True
            break
    if predicate_is_numerical(rule.conclusion, numerical_predicate):
        rule_with_numerical_in_conc+=1
        num=True
    if num:
        rule_with_numerical += 1
        num_f.write(str(rule)+"\n")
    else:
        symb_f.write(str(rule)+"\n")
    
print("Rule with numerical : ", rule_with_numerical)
print("Rule without numerical : ", len(res_parsed) - rule_with_numerical)
print("Rule with numerical in hypotheses : ", rule_with_numerical_in_hyp)
print("Rule with numerical in conclusion : ", rule_with_numerical_in_conc)

num_f.close()
symb_f.close()

Rule with numerical :  6281
Rule without numerical :  39882
Rule with numerical in hypotheses :  6281
Rule with numerical in conclusion :  6059


### Test the rules through test

In [12]:
data = open(root_data_FB+"train.tsv", "r")
f = open(root_data_FB+"train_rdflib.nt", "w")

for line in data:
    f.write(line.split("\n")[0]+" . \n")

data.close()
f.close()

In [13]:
data = open(root_source_FB+"test.txt", "r")

set_instances_to_predict = set()

for line in data:
    line_splited = line.split("\n")[0].split("\t")
    for i in range(len(line_splited)):
        line_splited[i] = "<http:"+line_splited[i]+">"
    set_instances_to_predict.add(tuple(line_splited))
        
data.close()

In [14]:
print(len(set_instances_to_predict))

59250


In [15]:
rules_associated_to_query = {}

for rule in res_parsed:
    if rule.conclusion.predicate in rules_associated_to_query.keys():
        rules_associated_to_query[rule.conclusion.predicate].add(rule)
    else:
        rules_associated_to_query[rule.conclusion.predicate] = set()
        rules_associated_to_query[rule.conclusion.predicate].add(rule)

In [16]:
g = rdflib.Graph()
g.parse(root_data_FB+"train_rdflib.nt")

<Graph identifier=N5346755583eb43cdbab91b5d0c8a2a7a (<class 'rdflib.graph.Graph'>)>

In [17]:
def print_rules_nicely(rule):
    toprint = ""
    for hyp in rule.hypotheses:
        toprint += hyp.predicate+" & "
    toprint = toprint[:-2]
    toprint += "=>"+rule.conclusion.predicate
    print(toprint)

In [18]:
%%time

def predict_instance(name, g, queue, prediction_per_instance_man, rules_associated_to_query, cpt, total_length, print_advancment, print_advancment_precise):
    
    print(f"Process n°{name} : Launched")
    
    while not queue.empty():
        
        instance = queue.get()
        
        dict_tp = {}
        
        if instance[1] in rules_associated_to_query.keys():
            for rule in rules_associated_to_query[instance[1]]:
                try:
                    qres = g.query(create_query(rule, instance[2]))

                    set_res = set()
                    bool_res = False
                    for row in qres:
                        bool_res = True
                        set_res.add(str(row.a))

                    if bool_res:
                        dict_tp[rule] = set_res
            
                except:
                    print(create_query(rule, instance[2]))
                    
            prediction_per_instance_man[instance] = dict_tp
            
        else:
            prediction_per_instance_man[instance] = {}
        
        cpt.value += 1
        if (cpt.value/total_length > print_advancment.value ):
            print(print_advancment.value*100+" %")
            print_advancment.value+=0.1
        if (cpt.value/total_length > print_advancment_precise.value):
            print(print_advancment_precise.value*100+" %")
            print_advancment_precise.value+=0.01
        
    print(f"Process n°{name} : Finished")   
    
q = Queue()
prediction_per_instance = {}

for instance in list(set_instances_to_predict):
    q.put(instance)

size_queue = q.qsize()

print("Queue finished")

with Manager() as manager:

    processes_to_create = multiprocessing.cpu_count()-3
    processes = list()

    prediction_per_instance_man = manager.dict()
    cpt = manager.Value("cpt",0)
    print_advancment = manager.Value("print_advancment", 0)
    print_advancment_precise = manager.Value("print_advancment_precise", 0.9)

    for name in range(processes_to_create):
        x = Process(target=predict_instance, args=(name, g, q, prediction_per_instance_man, rules_associated_to_query, cpt, size_queue, print_advancment, print_advancment_precise))
        processes.append(x)
        x.start()
        
    for index, process in enumerate(processes):
        process.join()
        
    print("Start Copy")
    prediction_per_instance = prediction_per_instance_man.copy()

Queue finished


Process n°0 : Launched

Process n°1 : Launched

Process n°2 : Launched

Process n°3 : Launched

...

  File "/mnt/beegfs/home/soulard/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


  File "<timed exec>", line 35, in predict_instance


TypeError: unsupported operand type(s) for +: 'int' and 'str'


Start Copy


CPU times: user 1.2 s, sys: 1.38 s, total: 2.57 s
Wall time: 1min


In [19]:
len(prediction_per_instance.keys())

77

In [20]:
df_prediction = {}

for prediction_instance in prediction_per_instance:
    df_rules = {}
    for rule in prediction_per_instance[prediction_instance]:
        df_rules[rule] = [set(prediction_per_instance[prediction_instance][rule]), rule.stdConfidence, rule.pcaConfidence]
    df_prediction[prediction_instance] = pd.DataFrame.from_dict(df_rules, orient="index", columns=["Prediction", "Std Confidence", "Pca Confidence"])

In [21]:
print("----- Democracy -----")
hit_at(df_prediction, democracy, 2)
hit_at(df_prediction, democracy, 5)
hit_at(df_prediction, democracy, 10)
hit_at(df_prediction, democracy, 1000)

print("----- Expert -----")
hit_at(df_prediction, expert, 2)
hit_at(df_prediction, expert, 5)
hit_at(df_prediction, expert, 10)
hit_at(df_prediction, expert, 1000)

----- Democracy -----


0.2077922077922078
0.3246753246753247


0.4155844155844156
0.8441558441558441
----- Expert -----


0.14285714285714285
0.16883116883116883
0.22077922077922077


0.2857142857142857


test

In [22]:
%%time

def predict_instance(name, g, instances, prediction_per_instance_man, rules_associated_to_query, cpt, total_length, print_advancment):
    
    print(f"Process n°{name} : Launched")
    
    for instance in instances:
        
        dict_tp = {}
        if instance[1] in rules_associated_to_query.keys():
            for rule in rules_associated_to_query[instance[1]]:
                try:
                    qres = g.query(create_query(rule, instance[2]))
                    print(create_query(rule, instance[2]))

                    set_res = set()
                    bool_res = False
                    for row in qres:
                        bool_res = True
                        set_res.add(str(row.a))

                    if bool_res:
                        dict_tp[rule] = set_res
            
                except:
                    print(create_query(rule, instance[2]))
                    
            prediction_per_instance_man[instance] = dict_tp
            
        else:
            prediction_per_instance_man[instance] = {}
        
        cpt.value += 1
        if (cpt.value/total_length > print_advancment.value):
            print(cpt)
            print_advancment.value+=0.1
        
    print(f"Process n°{name} : Finished")

prediction_per_instance_test = {}

size_queue = len(list(set_instances_to_predict)[:10])

with Manager() as manager:

    processes_to_create = 1#multiprocessing.cpu_count()-3
    processes = list()

    prediction_per_instance_man = manager.dict()
    cpt = manager.Value("cpt",0)
    print_advancment = manager.Value("print_advancment",0)
    
    instances_list = list(set_instances_to_predict)[:1]

    for name in range(processes_to_create):
        x = Process(target=predict_instance, args=(name, g, instances_list[int(np.floor(name*len(instances_list)/processes_to_create)): int(np.floor((name+1)*len(instances_list)/processes_to_create))], 
                                                   prediction_per_instance_man, rules_associated_to_query, cpt, size_queue, print_advancment))
        processes.append(x)
        x.start()
        
    for index, process in enumerate(processes):
        process.join()

    prediction_per_instance_test = prediction_per_instance_man.copy()

Process n°0 : Launched


    SELECT DISTINCT ?a
    WHERE {
?f <http:/award/award_nominee/award_nominations./award/award_nomination/award_nominee> <http:/m/02bh9> .
?a <http:/tv/tv_program/tv_producer./tv/tv_producer_term/producer> ?f .
}


    SELECT DISTINCT ?a
    WHERE {
<http:/m/02bh9> <http:/tv/tv_producer/programs_produced./tv/tv_producer_term/program> ?a .
?a <http:/tv/tv_program/recurring_writers./tv/tv_program_writer_relationship/writer> <http:/m/02bh9> .
}


    SELECT DISTINCT ?a
    WHERE {
<http:/m/02bh9> <http:/award/award_nominee/award_nominations./award/award_nomination/nominated_for> ?a .
<http:/m/02bh9> <http:/film/producer/film> ?a .
}


    SELECT DISTINCT ?a
    WHERE {
<http:/m/02bh9> <http:/film/writer/film> ?a .
}

...


    SELECT DISTINCT ?a
    WHERE {
?a <http:/award/award_nominated_work/award_nominations./award/award_nomination/award_nominee> <http:/m/02bh9> .
?a <http:/film/film/directed_by> <http:/m/02bh9> .
}


    SELECT DISTINCT ?a
    WHERE {
<http:/m/02bh9> <http:/award/award_nominee/award_nominations./award/award_nomination/nominated_for> ?a .
?a <http:/film/film/written_by> <http:/m/02bh9> .
}

Value('cpt', 1)

Process n°0 : Finished

CPU times: user 149 ms, sys: 133 ms, total: 282 ms
Wall time: 1.95 s


# DB15K

In [23]:
data = open(root_source_DB+"numericals.txt", "r")

numerical_predicate = set()

for predicate in data:
    numerical_predicate.add(predicate.split("\n")[0])
    
data.close()

In [24]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [25]:
def give_group(value, groups):
    for i, value_group in enumerate(groups):
        if value < value_group:
            return i
    return i+1

def write_file(X, f, groups):
    f.write(f"{X['Subject']}\t{X['Predicate']}\t{give_group(X['Object'], groups)}\n")

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [26]:
data = open(root_source_DB+"train.txt", "r")
f = open(store_data_DB, "w")

dic_predicate = {}

for line in data: 
    line_split = line.split("\n")[0].split(" ")
    if len(line_split) == 1:
        line_split = line.split("\n")[0].split("\t")
    if line_split[1] in numerical_predicate:
        line_split[2] = float(line_split[2]) 
        line_split = tuple(line_split)
        if line_split[1] in dic_predicate.keys():
            dic_predicate[line_split[1]].add(line_split)
        else : 
            dic_predicate[line_split[1]] = {line_split}
    else:
        f.write(line)
    
data.close()
f.close()

NameError: name 'store_data_DB' is not defined

In [ ]:
f = open(store_data_DB, "a")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    tp_df.apply(write_file, args=(f, tp_df_describe), axis=1)

f.close()

In [ ]:
res = check_output(f'java -jar ./../amie3.jar {store_data_DB}', shell=True)

res_parsed = parse_amie(res)

In [ ]:
len(res_parsed)

In [ ]:
res_parsed

### Number of numericals in the rules

In [ ]:
def predicate_is_numerical(atom, numerical_predicate):
    return atom.predicate in numerical_predicate

rule_with_numerical_in_hyp = 0
rule_with_numerical_in_conc = 0
rule_with_numerical = 0

for rule in res_parsed:
    num = False
    for hyp in rule.hypotheses:
        if predicate_is_numerical(hyp, numerical_predicate):
            rule_with_numerical_in_hyp+=1
            num = True
            break
    if predicate_is_numerical(rule.conclusion, numerical_predicate):
        rule_with_numerical_in_conc+=1
        num=True
    if num:
        rule_with_numerical += 1
    
print("Rule with numerical : ", rule_with_numerical)
print("Rule without numerical : ", len(res_parsed) - rule_with_numerical)
print("Rule with numerical in hypotheses : ", rule_with_numerical_in_hyp)
print("Rule with numerical in conclusion : ", rule_with_numerical_in_conc)